In [ ]:
!pip install folium==0.2.1

     |████████████████████████████████| 69 kB 3.7 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=63aec494a8c23d5995ffad73b8fb932c35e6a9625fab73a0e7b0043472476da9
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
!pip install dalle-pytorch --upgrade

     |████████████████████████████████| 1.4 MB 2.4 MB/s 
     |████████████████████████████████| 6.8 MB 40.3 MB/s 
     |████████████████████████████████| 1.7 MB 32.4 MB/s 
     |████████████████████████████████| 51 kB 113 kB/s 
     |████████████████████████████████| 3.5 MB 29.6 MB/s 
     |████████████████████████████████| 47 kB 4.1 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 14.0 MB 33.2 MB/s 
     |████████████████████████████████| 66 kB 4.8 MB/s 
     |████████████████████████████████| 138 kB 47.3 MB/s 
     |████████████████████████████████| 2.0 MB 34.7 MB/s 
     |████████████████████████████████| 843 kB 1.3 MB/s 
     |████████████████████████████████| 127 kB 48.8 MB/s 
     |████████████████████████████████| 527 kB 46.8 MB/s 
     |████████████████████████████████| 74 kB 3.2 MB/s 
     |████████████████████████████████| 596 kB 44.8 MB/s 
     |████████████████████████████████| 112 kB 45.3 MB/s 
     |███████████████████

In [ ]:
!pip install wandb
import wandb
!wandb login

     |████████████████████████████████| 1.7 MB 5.5 MB/s 
     |████████████████████████████████| 143 kB 29.0 MB/s 
     |████████████████████████████████| 180 kB 20.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=fae4992f5c94decfa40d390657be8ff7502d34f250db95bf997ce1305ca2a119
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
!gdown https://drive.google.com/uc?id=1IsZG5VOtMHXjJ3rvNZ5O2_JoOWXYLGvI
# https://drive.google.com/uc?id=11bpXgN6rpzhSCPF0KDR6KMNsPanbt3b9
# https://drive.google.com/uc?id=1dj1cGLEV0v18uGObxQ2PnN33ayLs3iwq

Downloading...
From: https://drive.google.com/uc?id=1IsZG5VOtMHXjJ3rvNZ5O2_JoOWXYLGvI
To: /content/botpeg.zip
100% 1.56M/1.56M [00:00<00:00, 159MB/s]


In [ ]:
!rm -rf botpeg # pegheads image-and-text-data
!unzip botpeg.zip # pegheads.zip image-and-text-data.zip

Archive:  botpeg.zip
   creating: botpeg/attributes/
 extracting: botpeg/attributes/certainties.txt  
  inflating: botpeg/attributes/class_attribute_labels_continuous.txt  
  inflating: botpeg/attributes/image_attribute_labels.txt  
  inflating: botpeg/bounding_boxes.txt  
  inflating: botpeg/classes.txt      
  inflating: botpeg/image_class_labels.txt  
  inflating: botpeg/images.txt       
   creating: botpeg/images/
   creating: botpeg/images/1.Dance/
  inflating: botpeg/images/1.Dance/Dance_01.jpg  
  inflating: botpeg/images/1.Dance/Dance_02.jpg  
   creating: botpeg/images/10.Singing/
  inflating: botpeg/images/10.Singing/Singing_01.jpg  
  inflating: botpeg/images/10.Singing/Singing_02.jpg  
  inflating: botpeg/images/10.Singing/Singing_03.jpg  
  inflating: botpeg/images/10.Singing/Singing_04.jpg  
  inflating: botpeg/images/10.Singing/Singing_05.jpg  
  inflating: botpeg/images/10.Singing/Singing_06.jpg  
  inflating: botpeg/images/10.Singing/Singing_07.jpg  
  inflating: botp

In [ ]:
import math
from math import sqrt

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR

# vision imports
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid, save_image

# dalle classes
from dalle_pytorch import DiscreteVAE

# constants
IMAGE_SIZE = 128
IMAGE_PATH = '/content/botpeg/images'

EPOCHS = 500
BATCH_SIZE = 4
LEARNING_RATE = 1e-3
LR_DECAY_RATE = 0.98

NUM_TOKENS = 8192
NUM_LAYERS = 2
NUM_RESNET_BLOCKS = 2
SMOOTH_L1_LOSS = False
EMB_DIM = 512
HID_DIM = 256
KL_LOSS_WEIGHT = 0

STARTING_TEMP = 1.
TEMP_MIN = 0.5
ANNEAL_RATE = 1e-6

NUM_IMAGES_SAVE = 4

# data
ds = ImageFolder(
    IMAGE_PATH,
    T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize(IMAGE_SIZE),
        T.CenterCrop(IMAGE_SIZE),
        T.ToTensor()
    ])
)

dl = DataLoader(ds, BATCH_SIZE, shuffle = True)

vae_params = dict(
    image_size = IMAGE_SIZE,
    num_layers = NUM_LAYERS,
    num_tokens = NUM_TOKENS,
    codebook_dim = EMB_DIM,
    hidden_dim   = HID_DIM,
    num_resnet_blocks = NUM_RESNET_BLOCKS
)

vae = DiscreteVAE(
    **vae_params,
    smooth_l1_loss = SMOOTH_L1_LOSS,
    kl_div_loss_weight = KL_LOSS_WEIGHT
).cuda()


assert len(ds) > 0, 'folder does not contain any images'
print(f'{len(ds)} images found for training')

def save_model(path):
    save_obj = {
        'hparams': vae_params,
        'weights': vae.state_dict()
    }

    torch.save(save_obj, path)

# optimizer
opt = Adam(vae.parameters(), lr = LEARNING_RATE)
sched = ExponentialLR(optimizer = opt, gamma = LR_DECAY_RATE)

# weights & biases experiment tracking
model_config = dict(
    num_tokens = NUM_TOKENS,
    smooth_l1_loss = SMOOTH_L1_LOSS,
    num_resnet_blocks = NUM_RESNET_BLOCKS,
    kl_loss_weight = KL_LOSS_WEIGHT
)

run = wandb.init(
    project = 'dalle_train_vae',
    job_type = 'train_model',
    config = model_config
)

# starting temperature
global_step = 0
temp = STARTING_TEMP

for epoch in range(EPOCHS):
    for i, (images, _) in enumerate(dl):
        images = images.cuda()

        loss, recons = vae(
            images,
            return_loss = True,
            return_recons = True,
            temp = temp
        )

        opt.zero_grad()
        loss.backward()
        opt.step()

        logs = {}

        if i % 100 == 0:
            k = NUM_IMAGES_SAVE

            with torch.no_grad():
                codes = vae.get_codebook_indices(images[:k])
                hard_recons = vae.decode(codes)

            images, recons = map(lambda t: t[:k], (images, recons))
            images, recons, hard_recons, codes = map(lambda t: t.detach().cpu(), (images, recons, hard_recons, codes))
            images, recons, hard_recons = map(lambda t: make_grid(t.float(), nrow = int(sqrt(k)), normalize = True, range = (-1, 1)), (images, recons, hard_recons))

            logs = {
                **logs,
                'sample images':        wandb.Image(images, caption = 'original images'),
                'reconstructions':      wandb.Image(recons, caption = 'reconstructions'),
                'hard reconstructions': wandb.Image(hard_recons, caption = 'hard reconstructions'),
                'codebook_indices':     wandb.Histogram(codes),
                'temperature':          temp
            }

            save_model(f'./vae.pt')
            wandb.save('./vae.pt')

            # temperature anneal
            temp = max(temp * math.exp(-ANNEAL_RATE * global_step), TEMP_MIN)

            # lr decay
            sched.step()

        if i % 10 == 0:
            lr = sched.get_last_lr()[0]
            print(epoch, i, f'lr - {lr:6f} loss - {loss.item()}')

            logs = {
                **logs,
                'epoch': epoch,
                'iter': i,
                'loss': loss.item(),
                'lr': lr
            }

        wandb.log(logs)
        global_step += 1

    # save trained model to wandb as an artifact every epoch's end
    model_artifact = wandb.Artifact('trained-vae', type = 'model', metadata = dict(model_config))
    model_artifact.add_file('vae.pt')
    run.log_artifact(model_artifact)

# save final vae and cleanup
save_model('./vae-final.pt')
wandb.save('./vae-final.pt')

model_artifact = wandb.Artifact('trained-vae', type = 'model', metadata = dict(model_config))
model_artifact.add_file('vae-final.pt')
run.log_artifact(model_artifact)

wandb.finish()

63 images found for training


wandb: Currently logged in as: gavincapriola (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/torchvision/utils.py:50: UserWarning: range will be deprecated, please use value_range instead.
  warnings.warn(warning)


0 0 lr - 0.000980 loss - 0.30546608567237854
0 10 lr - 0.000980 loss - 0.4175953269004822
1 0 lr - 0.000960 loss - 0.19407251477241516
1 10 lr - 0.000960 loss - 0.1429627239704132
2 0 lr - 0.000941 loss - 0.15376992523670197
2 10 lr - 0.000941 loss - 0.15679118037223816
3 0 lr - 0.000922 loss - 0.15407182276248932
3 10 lr - 0.000922 loss - 0.10308878868818283
4 0 lr - 0.000904 loss - 0.10628040879964828
4 10 lr - 0.000904 loss - 0.08862641453742981
5 0 lr - 0.000886 loss - 0.08285088837146759
5 10 lr - 0.000886 loss - 0.12013989686965942
6 0 lr - 0.000868 loss - 0.09798455238342285
6 10 lr - 0.000868 loss - 0.10497968643903732
7 0 lr - 0.000851 loss - 0.0727556049823761
7 10 lr - 0.000851 loss - 0.05901220440864563
8 0 lr - 0.000834 loss - 0.055832818150520325
8 10 lr - 0.000834 loss - 0.05361298844218254
9 0 lr - 0.000817 loss - 0.05534543842077255
9 10 lr - 0.000817 loss - 0.04819953441619873
10 0 lr - 0.000801 loss - 0.04685979336500168
10 10 lr - 0.000801 loss - 0.03842338919639587

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
iter,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,█▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
temperature,██████▇▇▇▇▆▆▆▅▅▄▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,499
iter,10
loss,0.0063
lr,0.0
temperature,0.5
